In [ ]:
!pip install transformers
from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification, AdamW

In [ ]:
import torch
import pandas as pd

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')

In [ ]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
df['tokenized_text'] = df['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))
df.head()

,id,keyword,location,text,target,tokenized_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[101, 3458, 9115, 3680, 1132, 1103, 21642, 110..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[101, 4089, 1783, 1485, 2001, 6413, 2176, 1778..."
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[101, 1398, 3159, 1455, 1106, 112, 7890, 1107,..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[101, 1492, 117, 1288, 1234, 3531, 108, 4098, ..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[101, 2066, 1400, 1850, 1142, 6307, 1121, 1137..."


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased")

optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

tokenized_texts = df['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, return_tensors='pt'))
labels = torch.tensor(df['target'].values)

epoch_number = 4 # @param {type:"slider", min:1, max:5, step:1}

for epoch in range(3):  # Choose the number of epochs
    for i in range(len(df)):
        optimizer.zero_grad()
        outputs = model(tokenized_texts[i], labels=torch.tensor([labels[i]]))
        loss = loss_fn(outputs.logits, torch.tensor([labels[i]]))
        loss.backward()
        optimizer.step()
    print(f"Epoch number {epoch} done!")
    torch.save(model.state_dict(), f"/content/drive/My Drive/model_{epoch}.pt")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/7613 [00:00<?, ?it/s]

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')

test_tokenized_texts = test_df['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, return_tensors='pt'))
test_labels = torch.tensor(test_df['target'].values)

predicted_labels = []
with torch.no_grad():
    for i in range(len(test_df)):
        outputs = model(test_tokenized_texts[i])
        predicted = torch.argmax(outputs.logits)
        predicted_labels.append(predicted.item())  # Append predicted labels to a list

# Compare predictions with the actual labels (if available)
if 'target' in test_df:
    correct_predictions = sum(predicted_labels[i] == test_labels[i] for i in range(len(test_df)))
    accuracy = correct_predictions / len(test_df)
    print(f"Test Set Accuracy: {accuracy}")